In [1]:
import geopandas as gpd
import pandas as pd
import shutil
import os

from utils import *

/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [28]:
# Clean coordinates
bol_ef = gpd.read_file("/Users/heatherbaier/Documents/geo_git/data/BOL/EstabEducativos/EstabEducativos.shp")
bol_ef


,gml_id,CODIGO,NOM_DEP,NOM_PROV,CAT_SEC,NOM_MUN,OBSERVACIO,COD_LE,CANTUE,POINT_X,POINT_Y,COD_PRIN,NOM_PRIN,COD_SIE,ESTABLECIM,geometry
0,EstabEducativos.1,060202,Tarija,Arce,SEGUNDA,Bermejo,None,71720016.0,1.0,-64.316993,-22.851395,71720022,NARANJITOS,71720022,NARANJITOS,POINT (-64.31699 -22.85140)
1,EstabEducativos.2,060202,Tarija,Arce,SEGUNDA,Bermejo,None,71720015.0,1.0,-64.310920,-22.806575,71720021,CAMPO GRANDE,71720021,CAMPO GRANDE,POINT (-64.31092 -22.80657)
2,EstabEducativos.3,060202,Tarija,Arce,SEGUNDA,Bermejo,None,71720017.0,1.0,-64.297661,-22.775036,71720023,PORCELANA,71720023,PORCELANA,POINT (-64.29766 -22.77504)
3,EstabEducativos.4,060202,Tarija,Arce,SEGUNDA,Bermejo,None,71720004.0,1.0,-64.337343,-22.741393,71720010,8 DE SEPTIEMBRE,71720010,8 DE SEPTIEMBRE,POINT (-64.33734 -22.74139)
4,EstabEducativos.5,060202,Tarija,Arce,SEGUNDA,Bermejo,None,0.0,0.0,-64.335151,-22.736408,None,None,None,None,POINT (-64.33515 -22.73641)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14732,EstabEducativos.14733,090502,Pando,Federico Roman,SEGUNDA,Villa Nueva (Loma Alta),None,0.0,0.0,-66.214088,-10.535879,None,None,None,None,POINT (-66.21409 -10.53588)
14733,EstabEducativos.14734,090502,Pando,Federico Roman,SEGUNDA,Villa Nueva (Loma Alta),None,62440018.0,1.0,-66.161590,-10.527191,62440018,19 DE OCTUBRE,62440018,19 DE OCTUBRE,POINT (-66.16159 -10.52719)
14734,EstabEducativos.14735,080202,Beni,Vaca Diez,SEGUNDA,Guayaramerín,None,82220096.0,1.0,-65.395085,-10.392153,82220114,VILLA BELLA,82220114,VILLA BELLA,POINT (-65.39509 -10.39215)
14735,EstabEducativos.14736,090401,Pando,Abuná,PRIMERA,Santa Rosa del Abuná,None,82450015.0,1.0,-67.180540,-10.340450,82450009,DEFENSORES DEL ABUNA,82450009,DEFENSORES DEL ABUNA,POINT (-67.18054 -10.34045)


In [4]:
bol_ef["CAT_SEC"].unique()

array(['SEGUNDA', 'PRIMERA', 'TERCERA', 'PRIMERA SECCION', 'PRIMERO',
       'SEPTIMA', 'SEXTA', 'SALAR', 'QUINTA', 'CAPITAL', 'CUARTA', None,
       'Primera', 'OCTAVA', 'LAGO'], dtype=object)

In [56]:
import requests

url = "https://seie.minedu.gob.bo:8443/geoserver/minedu/wfs"
params = {
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "minedu:vw_unidad_geo",  # try vw_unidad_geo2 or ue_servicios1 if needed
    "outputFormat": "application/json",
    # "count": 1000  # you can paginate with startIndex if there are many features
}

r = requests.get(url, params=params, verify=False)  # skip SSL verify for now
print(r.status_code)
data = r.json()
print(f"Returned {len(data['features'])} features")


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
Returned 16049 features


In [66]:
import geopandas as gpd

# Assuming `data` is your parsed JSON dict from r.json()
gdf = pd.DataFrame(gpd.GeoDataFrame.from_features(data["features"])).reset_index()
gdf.head()

,index,geometry,cod_pol_dep,des_dep,cod_pol_pro,des_pro,cod_pol_mun,des_sec,depend,area,cod_ue,cod_le,des_ue,cod_dis,des_dis,latitud,longitud
0,0,POINT (-7642489.085 -1829597.715),2,LA PAZ,202,OMASUYOS ...,20204,HUARINA ...,1,R,80720095,80720095,JUAN CHALCO,2091,None,-16.214683,-68.653648
1,1,POINT (-7196371.257 -2307405.349),1,CHUQUISACA,107,NOR CINTI ...,10702,SAN LUCAS ...,1,R,70420002,70420002,OJEDA,1020,SAN LUCAS ...,-20.289900,-64.646103
2,2,POINT (-7622964.985 -1969948.718),2,LA PAZ,203,PACAJES ...,20308,SANTIAGO DE CALLAPA ...,1,R,10710001,10710001,CANUTA,2013,SANTIAGO DE CALLAPA (C. CALLAPA) ...,-17.421525,-68.478260
3,3,POINT (-7664263.606 -1741737.494),2,LA PAZ,205,MUÑECAS ...,20502,AYATA ...,1,R,70690001,70690001,HUANCANIPAMPA,2018,AYATA ...,-15.455379,-68.849251
4,4,POINT (-7509020.785 -1859564.327),2,LA PAZ,211,SUD YUNGAS ...,21102,IRUPANA (VILLA DE LANZA) ...,1,R,70630001,70630001,5 DE MAYO,2041,IRUPANA (C. VILLA DE LANZA) ...,-16.473000,-67.454681


In [70]:
gdf["depend"].unique()

array(['1', '2'], dtype=object)

In [68]:
gdf.to_csv("../../data/BOL/bol_schools_from_wms.csv")

In [60]:
# import requests
# import pandas as pd
# from bs4 import BeautifulSoup

# school_id = "61880091"
# url = f"https://seie.minedu.gob.bo/reportes/mapas_unidades_educativas/ficha/ver/{school_id}"

# r = requests.get(url, verify=False)
# soup = BeautifulSoup(r.text, "html.parser")

# # --- 1. Extract "Nivel" ---
# # Find the <dt> with text "Nivel(es):" and get the next <dd>
# nivel_dt = soup.find("dt", string=lambda t: t and "Nivel(es)" in t)
# nivel = None
# if nivel_dt:
#     nivel_dd = nivel_dt.find_next_sibling("dd")
#     if nivel_dd:
#         nivel = nivel_dd.get_text(strip=True)

# print(f"Nivel(es): {nivel}")


# # --- 2. Extract the technical sheet table ---
# tables = pd.read_html(r.text)
# matricula = tables[0]
# promovidos = tables[1]
# reprobados = tables[2]
# abandono = tables[3]
# print(tables)
# # table_df = tables[0] if tables else pd.DataFrame()

# # # Combine results
# # print("DATOS GENERALES:")
# # for k, v in general_info.items():
# #     print(f"{k}: {v}")

# # print("\nTECHNICAL SHEET TABLE:")
# # print(table_df.head())


In [61]:
# reprobados

In [62]:
import pandas as pd
import requests

def scrape_school_long(school_id):
    url = f"https://seie.minedu.gob.bo/reportes/mapas_unidades_educativas/ficha/ver/{school_id}"
    r = requests.get(url, verify=False)

    soup = BeautifulSoup(r.text, "html.parser")

    # --- 1. Extract "Nivel" ---
    # Find the <dt> with text "Nivel(es):" and get the next <dd>
    nivel_dt = soup.find("dt", string=lambda t: t and "Nivel(es)" in t)
    nivel = None
    if nivel_dt:
        nivel_dd = nivel_dt.find_next_sibling("dd")
        if nivel_dd:
            nivel = nivel_dd.get_text(strip=True)

    print(nivel)



    # Parse all tables
    tables = pd.read_html(r.text)
    vars_ = ["matricula", "promovidos", "reprobados", "abandonos"]
    records = []

    for var, df in zip(vars_, tables):
        # Keep only relevant rows
        df = df[df["Sexo"].isin(["Total", "Mujer", "Hombre"])]
        year_cols = [c for c in df.columns if c.isdigit()]

        for year in year_cols:
            total_val = df.loc[df["Sexo"] == "Total", year].values[0]
            female_val = df.loc[df["Sexo"] == "Mujer", year].values[0] if "Mujer" in df["Sexo"].values else 0
            male_val = df.loc[df["Sexo"] == "Hombre", year].values[0] if "Hombre" in df["Sexo"].values else 0

            records.append({
                "school_id": school_id,
                "year": int(year),
                "variable": var,
                "total": total_val,
                "female": female_val,
                "male": male_val
            })

    df = pd.DataFrame(records)

    pivoted = df.pivot_table(
        index=["school_id", "year"], 
        columns="variable", 
        values=["total", "female", "male"]
    )

    # Flatten MultiIndex: becomes total_abandonos, female_promovidos, etc.
    pivoted.columns = [f"{var}_{col}" for col, var in pivoted.columns]
    pivoted = pivoted.reset_index()

    pivoted["nivel"] = nivel

    return pivoted




# Example usage
school_ids = gdf["cod_ue"].unique()[0:5]  # replace with your real IDs
for sid in school_ids:
    scrape_school_long(sid).to_csv(f"../../data/BOL/scrape/{sid}.csv")

# print(df.head())


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Inicial/Primaria/Secundaria


/var/folders/qv/hlnb1jfx68x_r2g75s1fkjlm0000gq/T/ipykernel_37864/206339849.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Inicial/Primaria


/var/folders/qv/hlnb1jfx68x_r2g75s1fkjlm0000gq/T/ipykernel_37864/206339849.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Inicial/Primaria


/var/folders/qv/hlnb1jfx68x_r2g75s1fkjlm0000gq/T/ipykernel_37864/206339849.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Inicial/Primaria/Secundaria


/var/folders/qv/hlnb1jfx68x_r2g75s1fkjlm0000gq/T/ipykernel_37864/206339849.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/var/folders/qv/hlnb1jfx68x_r2g75s1fkjlm0000gq/T/ipykernel_37864/206339849.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


Secundaria


In [51]:
df

,school_id,year,abandonos_female,matricula_female,promovidos_female,reprobados_female,abandonos_male,matricula_male,promovidos_male,reprobados_male,abandonos_total,matricula_total,promovidos_total,reprobados_total
0,61880091,2018,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
1,61880091,2019,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,61880091,2020,0.0,6.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,14.0,0.0,0.0
3,61880091,2021,NaN,4.0,4.0,0.0,NaN,8.0,8.0,0.0,NaN,12.0,12.0,0.0
4,61880091,2022,NaN,5.0,5.0,0.0,NaN,6.0,4.0,2.0,NaN,11.0,9.0,2.0
5,61880091,2023,NaN,6.0,6.0,NaN,NaN,5.0,5.0,NaN,NaN,11.0,11.0,NaN
6,61880091,2024,0.0,4.0,4.0,NaN,0.0,3.0,3.0,NaN,0.0,7.0,7.0,NaN
7,61880123,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,61880123,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,61880123,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pivoted = df.pivot_table(
    index=["school_id", "year"], 
    columns="variable", 
    values=["total", "female", "male"]
)

# Flatten MultiIndex: becomes total_abandonos, female_promovidos, etc.
pivoted.columns = [f"{var}_{col}" for col, var in pivoted.columns]
pivoted = pivoted.reset_index()

# # Rename to your desired schema
# rename_map = {
#     "abandonos_female": "dropouts_female",
#     "abandonos_male": "dropouts_male",
#     "total_abandonos": "dropouts_total",
#     "female_promovidos": "promoted_female",
#     "male_promovidos": "promoted_male",
#     "total_promovidos": "promoted_total",
#     "female_reprobados": "failed_female",
#     "male_reprobados": "failed_male",
#     "total_reprobados": "failed_total"
# }

# pivoted = pivoted.rename(columns=rename_map)

# # Keep only the mapped columns (school_id, year, and renamed ones)
# keep_cols = ["school_id", "year"] + list(rename_map.values())
# pivoted = pivoted[keep_cols]

pivoted.head()


,school_id,year,abandonos_female,matricula_female,promovidos_female,reprobados_female,abandonos_male,matricula_male,promovidos_male,reprobados_male,abandonos_total,matricula_total,promovidos_total,reprobados_total
0,61880091,2018,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
1,61880091,2019,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,61880091,2020,0.0,6.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,14.0,0.0,0.0
3,61880091,2021,NaN,4.0,4.0,0.0,NaN,8.0,8.0,0.0,NaN,12.0,12.0,0.0
4,61880091,2022,NaN,5.0,5.0,0.0,NaN,6.0,4.0,2.0,NaN,11.0,9.0,2.0


In [15]:
import requests

url = "https://seie.minedu.gob.bo:8443/geoserver/minedu/wfs"
params = {
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "minedu:vw_unidad_geo",
    "outputFormat": "application/json",
    "count": 1
}

r = requests.get(url, params=params, verify=False)
print(r.status_code)
print(r.headers.get("Content-Type"))
print(r.text[:500])  # preview first 500 characters


200
application/json;charset=UTF-8
{"type":"FeatureCollection","totalFeatures":16049,"features":[{"type":"Feature","id":"vw_unidad_geo.fid-3be85099_19996fbc72c_1745","geometry":{"type":"Point","coordinates":[-7642489.0854763,-1829597.71480648]},"geometry_name":"st_setsrid","properties":{"cod_pol_dep":2,"des_dep":"LA PAZ      ","cod_pol_pro":202,"des_pro":"OMASUYOS                                          ","cod_pol_mun":20204,"des_sec":"HUARINA                                                                                       


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
import requests

layers = [
    "minedu:vw_unidad_geo",
    "minedu:vw_unidad_geo4",
    "minedu:vw_unidad_geo5",
    "minedu:vw_unidad_geo6",
    "minedu:vw_unidad_geo7",
    "minedu:ue_servicios1",
    "minedu:ue_servicios2"
]

url = "https://seie.minedu.gob.bo:8443/geoserver/minedu/wfs"

for layer in layers:
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "typeNames": layer,
        "outputFormat": "application/json",
        "count": 1  # just fetch 1 feature to test
    }
    r = requests.get(url, params=params, verify=False)
    print(f"Layer: {layer} | Status: {r.status_code} | Content-Type: {r.headers.get('Content-Type')}")
    print(r.text[:200])  # preview first 200 chars
    print("-" * 80)


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Layer: minedu:vw_unidad_geo | Status: 200 | Content-Type: application/json;charset=UTF-8
{"type":"FeatureCollection","totalFeatures":16049,"features":[{"type":"Feature","id":"vw_unidad_geo.fid-3be85099_19996fbc72c_135a","geometry":{"type":"Point","coordinates":[-7642489.0854763,-1829597.7
--------------------------------------------------------------------------------


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Layer: minedu:vw_unidad_geo4 | Status: 200 | Content-Type: application/xml
<ows:ExceptionReport xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2.0.0" xsi:schemaLocation="ht
--------------------------------------------------------------------------------


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Layer: minedu:vw_unidad_geo7 | Status: 200 | Content-Type: application/json;charset=UTF-8
{"type":"FeatureCollection","totalFeatures":16099,"features":[{"type":"Feature","id":"vw_unidad_geo7.fid-3be85099_19996fbc72c_135c","geometry":{"type":"Point","coordinates":[-7196371.2571846,-2307405.
--------------------------------------------------------------------------------


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Layer: minedu:ue_servicios1 | Status: 200 | Content-Type: application/xml
<ows:ExceptionReport xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2.0.0" xsi:schemaLocation="ht
--------------------------------------------------------------------------------
Layer: minedu:ue_servicios2 | Status: 200 | Content-Type: application/json;charset=UTF-8
{"type":"FeatureCollection","totalFeatures":0,"features":[],"crs":null}
--------------------------------------------------------------------------------


/Users/heatherbaier/opt/anaconda3/envs/caoep/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seie.minedu.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
